In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')


In [3]:
df = df.set_index(pd.DatetimeIndex(df['Created Date']))
del df['Created Date']

## Question 1

In [4]:
df_2022 = df[df.index.year == 2022]

In [5]:
if not df_2022.empty:
        # Calculate the average number of daily complaints received in 2022
        daily_complaints_2022 = df_2022['Unique Key'].resample('D').count()
        average_daily_complaints_2022 = daily_complaints_2022.mean()

        print("Average number of daily complaints received in 2022:", round(average_daily_complaints_2022))

else:
        print("No data available for the year 2022.")

Average number of daily complaints received in 2022: 8684


## Question 2:


In [7]:
df.index = pd.to_datetime(df.index)


In [8]:
daily_calls = df.groupby(df.index.date).size()


In [9]:
max_calls_date = daily_calls.idxmax()

In [10]:
max_calls_date

datetime.date(2020, 8, 4)

## Question 3:


In [6]:
df.index = pd.to_datetime(df.index)


In [7]:
daily_calls = df.groupby(df.index.date).size()


In [8]:
max_calls_date = daily_calls.idxmax()

In [11]:
max_calls_df = df[df.index.date == max_calls_date]

In [12]:
most_important_complaint = max_calls_df['Complaint Type'].mode().iloc[0]

print(f"On the date with the maximum number of calls ({max_calls_date}), the most important complaint type was: {most_important_complaint}")

On the date with the maximum number of calls (1970-01-01), the most important complaint type was: Noise - Residential


## Question 4:

In [12]:
calls_per_month = df.resample('M').size()


In [13]:
quietest_month = calls_per_month.idxmin()


In [14]:
print(f"The quietest month, historically, is: {quietest_month.strftime('%B %Y')}")

The quietest month, historically, is: January 1970



## Question 5:

In [3]:
import pandas as pd
import statsmodels.api as sm

In [4]:
df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')

In [5]:
df.index = pd.to_datetime(df['Created Date'])
df = df.set_index('Created Date')


In [8]:
df_daily = df['Unique Key'].resample('D').count()

In [9]:
result = sm.tsa.seasonal_decompose(df_daily, model='additive')

In [10]:
seasonal_component = result.seasonal['2020-12-25'].round().astype(int)


In [11]:
print(f"The value of the seasonal component on 2020-12-25 (rounded to the nearest integer) is: {seasonal_component}")

The value of the seasonal component on 2020-12-25 (rounded to the nearest integer) is: 183


## Question 6:

In [12]:
autocorr_lag1 = df_daily.autocorr(lag=1)

In [13]:
print(f"The autocorrelation with a lag of 1 is: {autocorr_lag1}")

The autocorrelation with a lag of 1 is: 0.7517059728398577


## Question 7:

In [1]:
import pandas as pd
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df = pd.read_pickle('shared/Project-3_NYC_311_Calls.pkl')

In [4]:
df.index = pd.to_datetime(df['Created Date'])
df = df.set_index('Created Date')


In [5]:

df_daily = df['Unique Key'].resample('D').count().reset_index()
df_daily.columns = ['ds', 'y']

In [6]:
model = Prophet()
model.fit(df_daily)

01:29:22 - cmdstanpy - INFO - Chain [1] start processing
01:29:23 - cmdstanpy - INFO - Chain [1] done processing


In [9]:
future = model.make_future_dataframe(periods=90)

In [10]:
forecast = model.predict(future)

In [11]:
actual_values = df_daily.tail(90)['y'].values

In [12]:
predicted_values = forecast['yhat'][-90:].values

In [13]:
rmse = np.sqrt(mean_squared_error(actual_values, predicted_values))

In [14]:
print(f"The RMSE on the test set is: {rmse:.2f}")

The RMSE on the test set is: 1474.14
